# 영화진흥위원회 OpenAPI 연동

## #01. OpenAPI 

### [1] OpenAPI의 이해

누구나 사용할 수 있는 공개된 API

데이터를 표준화하고 프로그래밍해 외부 소프트웨어 개발자나 사용자가 바로 활용할 수 있는 형태의 개방 형식.

RESTFUL API에서 데이터 조회 기능만 제공하는 경우가 많다.

### [2] QueryString

다른 웹 프로그램에 URL을 통해서 전달하는 파라미터 문자열

```
https://www.naver.com/news?이름=값&이름=값...
```

웹 프로그램은 URL에 포함된 변수를 활용하여 하나의 프로그램이 변수값에 따라 다른 데이터를 표시할 수 있다.

### [3] JSON 데이터를 출력하는 웹 프로그램

어떤 웹 페이지가 URL에 포함된 파라미터를 통해 선택적으로 데이터를 표현할 수 있고, 이 웹페이지의 표현 형식이 JSON이라면 그 웹페이지는 파라미터에 따라 다른 내용을 제공할 수 있다.

## #02. 준비과정

### [1] 패키지 참조

In [1]:
import requests
import json
import datetime as dt

### [2] 접속할 데이터의 URL

변수값을 치환하기 위한 형식 문자열로 구성

In [2]:
urlFmt = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={targetDt}"

### [3] 요청 변수

#### (1) 발급받은 API 키

In [3]:
API_KEY = "6d2cf4aa96725383235c717f2e569f1e"

#### (2) 하루 전 날짜 얻기

In [4]:
date = dt.datetime.now() - dt.timedelta(days=1)
yesterday = date.strftime("%Y%m%d")
yesterday

'20231209'

## #03. 데이터 요청하기

### [1] 세션 생성

In [5]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

### [2] 데이터 가져오기

In [6]:
url = urlFmt.format(key=API_KEY, targetDt=yesterday)

try:
    r = session.get(url)
    
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)

### [3] 수신된 응답 결과 확인

In [7]:
r.encoding = "utf-8"
mydict = json.loads(r.text)
print(type(mydict))
mydict

<class 'dict'>


{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20231209~20231209',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20212866',
    'movieNm': '서울의 봄',
    'openDt': '2023-11-22',
    'salesAmt': '6305387146',
    'salesShare': '76.3',
    'salesInten': '3468244688',
    'salesChange': '122.2',
    'salesAcc': '61338908193',
    'audiCnt': '626195',
    'audiInten': '336262',
    'audiChange': '116',
    'audiAcc': '6387789',
    'scrnCnt': '2301',
    'showCnt': '9332'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20190324',
    'movieNm': '3일의 휴가',
    'openDt': '2023-12-06',
    'salesAmt': '526035962',
    'salesShare': '6.4',
    'salesInten': '161596027',
    'salesChange': '44.3',
    'salesAcc': '1558974957',
    'audiCnt': '53731',
    'audiInten': '15361',
    'audiChange': '40',
    'audiAcc': '166266',
    'scrnCnt': '863',
    '

## #04. 데이터 활용

> 데이터를 어떻게 활용하는지는 분석 목적에 따라 다양하게 구현될 수 있다.

In [8]:
fname = "박스오피스_%s.csv" % yesterday
with open(fname, "w", encoding="utf-8") as f:
    f.write("제목,개봉일,관람객수,누적관람객수\n")
    
    for item in mydict['boxOfficeResult']['dailyBoxOfficeList']:
        f.write("%s,%s,%s,%s\n" % (item['movieNm'], item['openDt'], item['audiCnt'], item['audiAcc']))
        
print("fin :)")

fin :)
